In [1]:
#install the libraries

In [2]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.7 MB/s eta 0:00:00


In [3]:
from collections import defaultdict
import gdown
import gzip
import json
import random
import pickle

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
sns.set(style='ticks', font_scale=1.2)

In [4]:
#import the libraries
import numpy as np
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#load and read the training TEXT From file
trainingSet = pd.read_csv('drive/MyDrive/germanOriginal.txt', on_bad_lines='skip', sep=';', header=None)

In [7]:
#show the head of the new CSV file
trainingSet.head()

,0
0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...


In [8]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
trainingSet=trainingSet.rename(columns={0: 'Tweet'})


In [9]:
trainingSet.head()

,Tweet
0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...


In [10]:
def check_for_abuse(row):
    if row.endswith("ABUSE"):
        return 1
    else:
        return 0



In [11]:
trainingSet['Label'] = trainingSet['Tweet'].apply(check_for_abuse)

In [12]:
trainingSet['Tweet'] = trainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [13]:
trainingSet['Tweet'] = trainingSet['Tweet'].str.replace('OTHER', '')
trainingSet['Tweet'] = trainingSet['Tweet'].str.replace('OFFENSE', '')

In [14]:
trainingSet.head()

,Tweet,Label
0,"@corinnamilborn Liebe Corinna, wir würden dich...",0
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,0
2,@ahrens_theo fröhlicher gruß aus der schönsten...,0
3,@dushanwegner Amis hätten alles und jeden gewä...,0
4,@spdde kein verläßlicher Verhandlungspartner. ...,0


In [15]:
trainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4811 entries, 0 to 4810
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4811 non-null   object
 1   Label   4811 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 75.3+ KB




**The Wiegand Corpus has now been loaded, and unnecessary data has been removed**
It contains 4811 tweets and labels with 0 or 1 (0 means no-hate speech, 1 signifies hate speech)





In [16]:
count = (trainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 995


In [17]:
countZero = (trainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 3816


In [18]:
trainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4811 entries, 0 to 4810
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4811 non-null   object
 1   Label   4811 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 75.3+ KB


**Processing and Cleaning the Tweets**

In [19]:
#import libraries for processing
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

1. Remove all links from the Tweets

In [20]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

trainingSet["tweet_w/o_url"] = trainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [21]:
trainingSet.head()

,Tweet,Label,tweet_w/o_url
0,"@corinnamilborn Liebe Corinna, wir würden dich...",0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...,0,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...,0,@spdde kein verläßlicher Verhandlungspartner. ...


In [22]:
trainingSet.head()

,Tweet,Label,tweet_w/o_url
0,"@corinnamilborn Liebe Corinna, wir würden dich...",0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...,0,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...,0,@spdde kein verläßlicher Verhandlungspartner. ...


In [23]:
X = trainingSet["tweet_w/o_url"].values.tolist()

In [24]:
y = trainingSet['Label'].values.tolist()

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [26]:
train_texts = X_train
train_labels = y_train

test_texts = X_test
test_labels = y_test

In [27]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(3848, 3848, 963, 963)

In [28]:
train_labels[0], train_texts[0]

(0, '@UlrichHenning2 @weissrot aber darum geht es doch gar nicht ')

In [29]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [30]:
#import svm model and metrics
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [31]:
#model = LogisticRegression(max_iter=1000).fit(X_train, train_labels)
#SVM BASE LINE
#model = SVC(kernel='linear', C=100, gamma='auto').fit(X_train, train_labels)
model = SVC(kernel='linear', gamma='auto').fit(X_train, train_labels)
predictions = model.predict(X_test)

In [32]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.82      0.98      0.89       761
           1       0.73      0.18      0.29       202

    accuracy                           0.81       963
   macro avg       0.78      0.58      0.59       963
weighted avg       0.80      0.81      0.77       963



In [33]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predictions)

0.2868525896414343

In [34]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predictions)

0.2985292264370886

Repeat the same with GermanBERT and GermanData

In [35]:
# This is the name of the BERT model that we want to use.
# We're using DistilBERT to save space (it's a distilled version of the full BERT model),
# and we're going to use the cased (vs uncased) version.
model_name = 'dbmdz/bert-base-german-cased'

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'MasterGermanBERT-model-saved'

In [36]:
from transformers import BertForSequenceClassification, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

In [37]:
train_labels

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,


In [38]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [39]:
label2id.keys()

dict_keys([0, 1])

In [40]:
id2label.keys()

dict_keys([0, 1])

In [41]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels]
test_labels_encoded  = [label2id[y] for y in test_labels]

In [42]:
set(train_labels_encoded)

{0, 1}

In [43]:
set(test_labels_encoded)

{0, 1}

In [44]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [45]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [46]:
# The model_name needs to match the name used for the tokenizer above.
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initi

In [47]:
#!pip install --upgrade accelerate

In [48]:
training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [49]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [50]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [51]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.513900,0.455852,0.789200
200,0.441800,0.466774,0.804777
300,0.388200,0.447570,0.791277
400,0.337700,0.361921,0.841121
500,0.281900,0.505719,0.831776
600,0.151400,0.628165,0.840083
700,0.195000,0.551364,0.839045
800,0.087700,0.783626,0.849429
900,0.051400,0.752239,0.840083


TrainOutput(global_step=964, training_loss=0.25778944685251387, metrics={'train_runtime': 241.7808, 'train_samples_per_second': 63.661, 'train_steps_per_second': 3.987, 'total_flos': 1668962757471360.0, 'train_loss': 0.25778944685251387, 'epoch': 4.0})

In [52]:
trainer.save_model(cached_model_directory_name)

In [53]:
trainer.evaluate()

{'eval_loss': 0.7299982309341431,
 'eval_accuracy': 0.8473520249221184,
 'eval_runtime': 3.7835,
 'eval_samples_per_second': 254.527,
 'eval_steps_per_second': 12.951,
 'epoch': 4.0}

In [54]:
predicted_results = trainer.predict(test_dataset)

In [55]:
predicted_results.predictions.shape

(963, 2)

In [56]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [57]:
len(predicted_labels)

963

Results for GermanBERT and GermanData

In [58]:
print(classification_report(test_labels,
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91       761
           1       0.66      0.57      0.61       202

    accuracy                           0.85       963
   macro avg       0.77      0.75      0.76       963
weighted avg       0.84      0.85      0.84       963



In [59]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predicted_labels)

0.610079575596817

In [60]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predicted_labels)

0.5178361907946922